In [2]:
%%time

import os

import numpy as np
import pandas as pd

from datetime import datetime, timedelta 

import math

# To plot graphs
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib nbagg

# To allow more outputs in Jupyter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Need to parse date with extra function because of the format
mydateparser = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S:%f')

# Path to left wrist, right wrist Actigraphy and Polysomnography data
path_ACG = "C:\\Users\\sigmu\\Desktop\\dataset\\" + "Aktigrafie\\"
path_PSG = "C:\\Users\\sigmu\\Desktop\\dataset\\" + "Polysomnografie\\"
files_ACG = os.listdir(path_ACG)
files_PSG = os.listdir(path_PSG)

# l_ACG_file = "test"
# r_ACG_file = "MECSLEEP60_right_wrist_018141_2015-05-21_15-32-02"
# PSG_file = "mecsleep60_psg.txt"

#l_ACG_path = path + "Aktigrafie\\" + l_ACG_file + '.csv'

# Read files, name columns, parse time stamp to date & put into index, skip rows to actual data
#df = pd.read_csv(path + "Aktigrafie\\" + ACG_file + '.csv', names=[
#                 'time stamp', 'x axis [g]', 'y axis [g]', 'z axis [g]', 'light level [lux]', 'button [1/0]','temperature [°C]'], 
#                 skiprows=100)
#df['time stamp'] = pd.to_datetime(df['time stamp'], format='%Y-%m-%d %H:%M:%S:%f')
#
#df_PSG = pd.read_csv(path + "Polysomnografie\\" + PSG_file + '.txt', sep='\t', skiprows=17)
#df_PSG['Time [hh:mm:ss]'] = pd.to_datetime(df_PSG['Time [hh:mm:ss]'], format='%H:%M:%S')
#
## Drop not needed columns
#df.drop(columns=['light level [lux]', 'button [1/0]', 'temperature [°C]'], inplace=True, axis=1)
#df_PSG.drop(columns=['Position', 'Event', 'Duration[s]'], inplace=True, axis=1)

Wall time: 7 ms


In [3]:
def comp_angle(row):
                return math.degrees(math.atan(row['z axis [g]'] / (row['x axis [g]']**2 + row['y axis [g]']**2)))
    
def inactiv(thr_1st, thr, clmn_name, df):
                counter = 1
                first_S = False
                for index, value in df['abs angle change'].items():
                    counter += 1

                    # Angle change > 5
                    if (value > 5):
                        counter = 0
                        df.loc[index, clmn_name] = "W"
                    # After first sleep - shorter threshold    
                    elif (counter > thr_1) & first_S:
                        counter += 1
                        df.loc[index, clmn_name] = "S"
                        SE = index
                    # First sleep - longer threshold 
                    elif (counter > thr_0):
                        counter += 1
                        df.loc[index, clmn_name] = "S"
                        first_S = True
                        # Sleep Onset (start of sleep)
                        SO = index
                return
            
def stats(match, state, df):    
                print(state)
                print("========")
                #  Returns relative frequency by dividing all values by the sum of values
                print("Relative frequency: ", df[match].value_counts(normalize='True'))
                print("True count: ", df[match].values.sum())
                print("False count: ", (~df[match]).values.sum())
                print("--------")

                # Count Total Sleep Time and Wake After Sleep Onset in minutes... so far all 'S' and 'W' states
                TST = (len(df[df[state]=='S'].index.value_counts()) * 30 ) / 60
                print("TST =", TST)
                df_stats['TST'][state] = TST

                WASO = (len(df[df[state]=='W'].index.value_counts()) * 30 ) / 60
                print("WASO =", WASO)
                df_stats['WASO'][state] = WASO

                # Sleep Wake Ratio ... the bigger the better sleeper 
                SWR = round(TST / WASO, 4)
                print("SWR =", SWR)
                df_stats['SWR'][state] = SWR
                print("--------")

                #sensitivity (actigraphy = sleep when PSG = sleep), 
                #specificity (actigraphy = wake when PSG = wake), 
                #and accuracy (total proportion correct)

                # Put all states where: into an array
                TP = np.where((df[state] == 'W') & (df['State PSG'] == 'W'))
                FP = np.where((df[state] == 'W') & (df['State PSG'] == 'S'))
                TN = np.where((df[state] == 'S') & (df['State PSG'] == 'S'))
                FN = np.where((df[state] == 'S') & (df['State PSG'] == 'W'))

                # Return array size
                TP_cnt = np.size(TP)
                FP_cnt = np.size(FP)
                TN_cnt = np.size(TN)
                FN_cnt = np.size(FN)

                # Calculate sensitivity, specificity and accuracy
                sens = round((TP_cnt / (TP_cnt + FN_cnt) )*100, 4)
                spec = round((TN_cnt / (TN_cnt + FP_cnt) )*100, 4)
                acc = round(((TP_cnt + TN_cnt) / (TP_cnt + TN_cnt + FP_cnt + FN_cnt) )*100, 4)

                print("sens =", sens)
                print("spec =", spec)
                print("acc =", acc)
                print("\n")

                # Add to df
                df_stats['Participant'][state] = participant
                df_stats['sensitivity'][state] = sens
                df_stats['specificity'][state] = spec    
                df_stats['accuracy'][state] = acc

                return

In [6]:
counter = 0
for file_name_PSG in files_PSG:
    participant = file_name_PSG.split("_")[0]
    for file_name_ACG in files_ACG:        
        if((file_name_ACG.startswith(participant)) & (counter <= 6)):
            counter += 1
            counter
            df = pd.read_csv(path_ACG+file_name_ACG, names=[
                 'time stamp', 'x axis [g]', 'y axis [g]', 'z axis [g]', 'light level [lux]', 'button [1/0]','temperature [°C]'], 
                 skiprows=100)
            df['time stamp'] = pd.to_datetime(df['time stamp'], format='%Y-%m-%d %H:%M:%S:%f')
            
            df_PSG = pd.read_csv(path_PSG+file_name_PSG, sep='\t', skiprows=17)
            df_PSG['Time [hh:mm:ss]'] = pd.to_datetime(df_PSG['Time [hh:mm:ss]'], format='%H:%M:%S')
            
            df.drop(columns=['light level [lux]', 'button [1/0]', 'temperature [°C]'], inplace=True, axis=1)
            df_PSG.drop(columns=['Position', 'Event', 'Duration[s]'], inplace=True, axis=1)                        
            
            #-----------------------------------------------------------------------------------------------
            # new df for plotting and comparing
            df_drp = df

            #def cutToMatch(df, df_PSG):
            df_PSG_len = len(df_PSG.index)-1
            df_len = len(df.index)-1

            # Match start
            # If df starts earlier than df_PSG -> cut start of df
            if(df['time stamp'].dt.hour[0] < df_PSG['Time [hh:mm:ss]'].dt.hour[0]):
                # Find df index where time matches
                idx_start = df[(df['time stamp'].dt.hour == df_PSG['Time [hh:mm:ss]'].dt.hour[0]) &
                               (df['time stamp'].dt.minute == df_PSG['Time [hh:mm:ss]'].dt.minute[0]) &
                               (df['time stamp'].dt.second == df_PSG['Time [hh:mm:ss]'].dt.second[0])].index[0]
                # Drop df from 0 to idx_start
                df_drp = df.drop(df.index[0:(idx_start)])#, inplace=True) 
                print('1')
                df_drp
            # Else cut start of df_PSG
            else:
                # Find df_PSG index where time matches
                idx_start = df_PSG[(df['time stamp'].dt.hour[0] == df_PSG['Time [hh:mm:ss]'].dt.hour) & 
                                   (df['time stamp'].dt.minute[0] == df_PSG['Time [hh:mm:ss]'].dt.minute)].index[0]
                # Drop df_PSG from 0 to idx_start
                df_PSG.drop(df_PSG.index[0:(idx_start)], inplace=True) 
                print('2')
                df_PSG

            # Match end
            # If df ends earlier than df_PSG -> cut end of df_PSG
            if(df['time stamp'].dt.hour[df_len] < df_PSG['Time [hh:mm:ss]'].dt.hour[df_PSG_len]):
                idx_end = df_PSG[(df['time stamp'].dt.hour[df_len] == df_PSG['Time [hh:mm:ss]'].dt.hour) &
                                 (df['time stamp'].dt.minute[df_len]+1 == df_PSG['Time [hh:mm:ss]'].dt.minute)].index[0]  # + 1 min  
                # Drop df_PSG from df end to end
                df_PSG.drop(df_PSG.index[(idx_end):(len(df_PSG.index))], inplace=True)
                print('3')
                df_PSG
            # Else cut end of df
            else:
                idx_end = df[(df['time stamp'].dt.hour == df_PSG['Time [hh:mm:ss]'].dt.hour[df_PSG_len]) &
                             (df['time stamp'].dt.minute == df_PSG['Time [hh:mm:ss]'].dt.minute[df_PSG_len]) &
                             (df['time stamp'].dt.second == df_PSG['Time [hh:mm:ss]'].dt.second[df_PSG_len])].index[0]
                # Drop df from df_PSG end to end
                df_drp.drop(df.index[(idx_end):(len(df.index))], inplace=True)
                print('4')
                df_drp
            
            #df = cutToMatch(df, df_df_PSG)
            #-----------------------------------------------------------------------------------------------
            # Resample by 5 second epoch and compute median of x,y,z

            # Set origin from PSG to start resampling
            orig = (df_PSG['Time [hh:mm:ss]'].dt.hour[0]*3600 + 
                    (df_PSG['Time [hh:mm:ss]'].dt.minute[0]*60) +
                    (df_PSG['Time [hh:mm:ss]'].dt.second[0]))
            orig  = pd.Timestamp(orig, unit='s')

            df_res = df_drp.resample('5S', on='time stamp', kind='timestamp', origin=orig).median().round(decimals=4)
            #-----------------------------------------------------------------------------------------------            
            # Apply func comp_angle
            df_res['angle'] = df_res.apply(comp_angle, axis=1).round(decimals=4)

            #Average angle per 5 sec epoch
            #df_res['angle'] = df_drp.resample('5S').mean().round(decimals=4)?????

            # Return absolute difference in angle per column
            df_res['abs angle change'] = df_res['angle'].diff().round(decimals=4).abs()

            # New column all "W" 
            state_5_10_1 = 'State 5s_10,1'
            state_5_10_5 = 'State 5s_10,5'

            df_res[state_5_10_1] = "W"
            df_res[state_5_10_5] = "W"

            state_30_10_1 = 'State 30s_10,1'
            state_30_10_5 = 'State 30s_10,5'
            #-----------------------------------------------------------------------------------------------        
            # (10 min)(1 min)
            thr_0 = (10 * 60) / 5
            thr_1 = (1 * 60) / 5
            inactiv(thr_0, thr_1, state_5_10_1, df_res)

            # (10 min)(5 min)
            thr_1 = (5 * 60) / 5
            inactiv(thr_0, thr_1, state_5_10_5, df_res)
            #-----------------------------------------------------------------------------------------------
            # Resample to have same number of columns (30s epochs)
            df_res_PSG = df_res.resample('30S', origin=orig).interpolate()

            # If df_PSG has extra value at the end
            #if(df_PSG['Time [hh:mm:ss]'].dt.second[len(df_PSG)-1] > df_res_PSG
            if(len(df_PSG) > len(df_res_PSG)):
                df_PSG.drop(df_PSG.index[len(df_PSG)-1], inplace=True)

            # Overwrite State PSG to bi-state 
            df_res_PSG['State PSG'] = np.where((df_PSG['Sleep Stage'] == 'N1')|(df_PSG['Sleep Stage'] == 'N2')|
                                             (df_PSG['Sleep Stage'] == 'N3')|(df_PSG['Sleep Stage'] == 'R'), 'S', 'W')

            # Create boolean column and compare
            match = 'match_' + state_5_10_1
            df_res_PSG[match] = np.where(df_res_PSG[state_5_10_1] == df_res_PSG['State PSG'], True, False)
            match = 'match_' + state_5_10_5
            df_res_PSG[match] = np.where(df_res_PSG[state_5_10_5] == df_res_PSG['State PSG'], True, False)
            #-----------------------------------------------------------------------------------------------
            df_stats = pd.DataFrame(columns=['Participant', 'TST', 'WASO', 'SWR', 'specificity', 'sensitivity', 'accuracy'],
                       index=[state_5_10_1, state_5_10_5, state_30_10_1, state_30_10_5])
    
            df_stats['Participant'][0] = PSG_file

            

            print("df_res_PSG")
            match = 'match_' + state_5_10_1
            stats(match, state_5_10_1, df_res_PSG)

            match = 'match_' + state_5_10_5
            stats(match, state_5_10_5, df_res_PSG)
            #-----------------------------------------------------------------------------------------------
            path = "C:\\Users\\sigmu\\Desktop\\dataset\\"
            returns_path_csv = path + "\\Results\\" + "Stats_iterate_" + '.csv'
            with open(returns_path_csv, 'a') as f:
                         (df_stats).to_csv(f)#, header=False)
            
            del df
            del df_PSG
